In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

In [ ]:
!nvcc --version

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
import cv2
cv2.__version__

'4.1.2'

In [ ]:
import time
time.asctime()

'Mon Nov  9 07:23:25 2020'

In [ ]:
%%cu
#include<stdio.h>
__global__ void add(int a,int b,int *c)
{
    *c=a+b;
}
int main(void)
{
    int c;
 int *dev_c;
 cudaMalloc((void**)&dev_c, sizeof(int));
 add << <1, 1 >> > (2, 7, dev_c);
 cudaMemcpy(&c, dev_c, sizeof(int),
 cudaMemcpyDeviceToHost);
 printf("output of 18bce1005 sruti verma\n");
 printf("2 + 7 = %d\n", c);
 cudaFree(dev_c);
 return 0;
}

output of 18bce1005 sruti verma
2 + 7 = 9



In [ ]:
%%cu
#include<stdio.h>
// adding two vectors 
__global__ void vector_add(int *out_d,int *a,int *b,int n)
{
    for(int i=0;i<n;i++)
    {out_d[i]=a[i]+b[i];}
}
int main()
{
    int *a,*b,*out_d,*out;
 int *d_a,*d_b;
 int N=64;
 int i;
 a=(int*)malloc(sizeof(int)*N);
 b=(int*)malloc(sizeof(int)*N);
 out=(int*)malloc(sizeof(int)*N);
 for(i=0;i<N;i++)
 {
     a[i]=i;
  b[i]=i*2;
 }
 cudaMalloc((void**)&d_a,sizeof(int)*N);
 cudaMalloc((void**)&d_b,sizeof(int)*N);
 cudaMalloc((void**)&out_d,sizeof(int)*N);
 cudaMemcpy(d_a,a,sizeof(int)*N,cudaMemcpyHostToDevice);
 cudaMemcpy(d_b,b,sizeof(int)*N,cudaMemcpyHostToDevice);
 vector_add<<<16,4>>>(out_d,d_a,d_b,N);
 cudaMemcpy(out,out_d,sizeof(int)*N,cudaMemcpyDeviceToHost);
 printf("output of 18bce1005 sruti verma\n");
 printf("Success\n");
 for(i=0;i<N;i++)
 {
     printf("%d\n",out[i]);
 }
 cudaFree(d_a);
 cudaFree(out_d);
 cudaFree(out_d);
 free(a);
 free(b);
 free(out);
 return 0;
}

output of 18bce1005 sruti verma
Success
0
3
6
9
12
15
18
21
24
27
30
33
36
39
42
45
48
51
54
57
60
63
66
69
72
75
78
81
84
87
90
93
96
99
102
105
108
111
114
117
120
123
126
129
132
135
138
141
144
147
150
153
156
159
162
165
168
171
174
177
180
183
186
189



In [ ]:
%%cu  
#include<stdio.h>  
// reverse of vector
__global__ void vector_add(int *out_d,int *a,int n)  
{  
    int id=blockIdx.x * blockDim.x+threadIdx.x;
 if(id<n)
    out_d[n-id-1]=a[id];
}  
int main()  
{  
    int *a,*out_d,*out;  
 int *d_a;  
 int N=10;  
 int i;  
 a=(int*)malloc(sizeof(int)*N);  
 out=(int*)malloc(sizeof(int)*N);  
 for(i=0;i<N;i++)  
 {  
     a[i]=i;  
 }  
 cudaMalloc((void**)&d_a,sizeof(int)*N);  
 cudaMalloc((void**)&out_d,sizeof(int)*N);  
 cudaMemcpy(d_a,a,sizeof(int)*N,cudaMemcpyHostToDevice);  
 vector_add<<<5,2>>>(out_d,d_a,N);  
 cudaMemcpy(out,out_d,sizeof(int)*N,cudaMemcpyDeviceToHost);  
printf("output of 18bce1005 sruti verma\n");
 printf("Success\n");  
 int k=0;
 for(i=0;i<N;i++)  
 { 
     printf("%d\n",out[i]); 
 }  
 cudaFree(d_a);  
  cudaFree(out_d);  
 free(a);  
 free(out);  
 return 0;  
}   

output of 18bce1005 sruti verma
Success
9
8
7
6
5
4
3
2
1
0



In [ ]:
%%cu 
#include<stdio.h>  
// two vectors sum
__global__ void arradd(int *x,int *y, int *z)  
{  
int id=blockIdx.x * blockDim.x+threadIdx.x;  
/* blockIdx.x gives the respective block id which starts from 0 */  
/* threadIdx.x gives the respective thread id which starts from 0 */  
/* blockDim.x gives the dimension of block i.e. number of threads in one block */  
    z[id]=x[id]+y[id];  
}  
int main()  
{  
    int a[6];  
    int b[6];  
    int c[6];  
    int *d,*e,*f;  
    int i;  
    for(i=0;i<6;i++)  
    {  
        a[i]=i;
    }  
        for(i=0;i<6;i++)  
        {  
           b[i]=i;
        }  
    cudaMalloc((void **)&d,6*sizeof(int));  
    cudaMalloc((void **)&e,6*sizeof(int));  
    cudaMalloc((void **)&f,6*sizeof(int));  
 cudaMemcpy(d,a,6*sizeof(int),cudaMemcpyHostToDevice);  
 cudaMemcpy(e,b,6*sizeof(int),cudaMemcpyHostToDevice);  
    arradd<<<2,3>>>(d,e,f);  
 cudaMemcpy(c,f,6*sizeof(int),cudaMemcpyDeviceToHost);  
    printf("\nSum of two arrays:\n ");  
    for(i=0;i<6;i++)  
    {  
        printf("%d\t",c[i]);  
    }  
    cudaFree(d);  
    cudaFree(e);  
    cudaFree(f);  
    return 0;  
} 

 

In [ ]:
%%cu    
#include<stdio.h>    
// generating multiples of n numbers by id method
__global__ void mul(int *f,int k)      
{      
    int id=blockIdx.x * blockDim.x+threadIdx.x;  
    f[id]=k*(id+1);   
}  
int main()    
{    
 int a[4];    
 int N=4,m=4;    
 int i;    
 for(i=0;i<N;i++)    
 {    
     a[i]=i+1;    
 }    
printf("output of 18bce1005 sruti verma\n");  
 printf("Success\n");    
 for(i=0;i<N;i++)    
 {    
     printf("\nmultiple of %d: ",a[i]);    
  int b[4];int *out_d;
   cudaMalloc((void **)&out_d,4*sizeof(int));  
 mul<<<2,2>>>(out_d,a[i]);      
 cudaMemcpy(b,out_d,4*sizeof(int),cudaMemcpyDeviceToHost);     
  for(int i=0;i<4;i++)
   printf("%d ",b[i]);
  cudaFree(out_d);  
 }    
 
 return 0;    
}     

output of 18bce1005 sruti verma
Success

multiple of 1: 1 2 3 4 
multiple of 2: 2 4 6 8 
multiple of 3: 3 6 9 12 
multiple of 4: 4 8 12 16 


In [ ]:
%%cu  
#include<stdio.h>  
#include<cuda.h>  
// three vectors multiplication 
cudaEvent_t start, stop; float elapsed_time_ms; 
__global__ void arradd(int *x,int *y,int *z,int *w)  
{  
 int id=blockIdx.x * blockDim.x+threadIdx.x; 
  w[id]=x[id]*y[id]*z[id];  
}  
int main()  
{  
    int a[4],b[4],c[4],r[4];  
    int *d,*e,*f,*g;  
    int i;  
    for(i=0;i<4;i++)  
    {  
        a[i] = i+1;  
    }  
    for(i=0;i<4;i++)  
    {  
        b[i] = i+1;  
    }  
    for(i=0;i<4;i++)  
    {  
        c[i] = i+1;  
    }  
    printf("\nInitial vectors:\n ");  
    for(i=0;i<4;i++)  
    {  
        printf("%d\t%d\t%d\n",a[i],b[i],c[i]);  
    }  
 cudaEventCreate(&start); cudaEventCreate(&stop);  
    cudaMalloc((void **)&d,4*sizeof(int));  
    cudaMalloc((void **)&e,4*sizeof(int));  
    cudaMalloc((void **)&f,4*sizeof(int));  
    cudaMalloc((void **)&g,4*sizeof(int));  
    cudaMemcpy(d,a,4*sizeof(int),cudaMemcpyHostToDevice);  
    cudaMemcpy(e,b,4*sizeof(int),cudaMemcpyHostToDevice);  
    cudaMemcpy(f,c,4*sizeof(int),cudaMemcpyHostToDevice);  
 cudaEventRecord(start, 0);  
    arradd<<<6,1>>>(d,e,f,g);   
 cudaEventRecord(stop, 0); 
    cudaMemcpy(r,g,4*sizeof(int),cudaMemcpyDeviceToHost);  
 cudaEventElapsedTime(&elapsed_time_ms, start, stop ); 
    printf("output of 18bce1005 sruti verma\n");  
    printf("\nFinal vector:\n ");  
    for(i=0;i<4;i++)  
    {  
        printf("%d\n",r[i]);  
    }  
 printf("\n Time Elapsed is - %f ms",elapsed_time_ms);  
    cudaFree(d);  
    cudaFree(e);  
    cudaFree(f);  
    cudaFree(g);  
    return 0;  
} 

In [ ]:
%%cu      
#include<stdio.h>      
// multiplying constant to vector
__global__ void vector_mul(int *out_d,int *a,int n,int k)      
{      
    int id=blockIdx.x * blockDim.x+threadIdx.x;    
    out_d[id]=a[id]*k;    
}      
int main()     
{      
    int *a,*out_d,*out;     
 int *d_a;      
 int N=10;     
 int i;      
 a=(int*)malloc(sizeof(int)*N);      
 out=(int*)malloc(sizeof(int)*N);      
 for(i=0;i<N;i++)     
 {      
     a[i]=i;      
 }      
 cudaMalloc((void**)&d_a,sizeof(int)*N);      
 cudaMalloc((void**)&out_d,sizeof(int)*N);      
 cudaMemcpy(d_a,a,sizeof(int)*N,cudaMemcpyHostToDevice);      
int k=2;    
 vector_mul<<<2,5>>>(out_d,d_a,N,k);      
 cudaMemcpy(out,out_d,sizeof(int)*N,cudaMemcpyDeviceToHost);      
printf("output of 18bce1005 sruti verma\n");  
 printf("Success\n");      
 for(i=0;i<N;i++)      
 {      
      a[i]=out[i];    
     printf("%d\n",a[i]);      
 }      
 cudaFree(d_a);      
  cudaFree(out_d);      
 free(a);      
 free(out);      
 return 0;      
}      

output of 18bce1005 sruti verma
Success
0
2
4
6
8
10
12
14
16
18



In [ ]:
%%cu        
#include<stdio.h>     
// finding index at which element is present   
__global__ void  find(int *out_d,int *i,int k)        
{        
    int id=blockIdx.x * blockDim.x+threadIdx.x;      
    if(out_d[id]==k)  
       i[0]=id;      
}        
int main()       
{        
    int *a,*d,*l;       
 int *d_a;        
 int N=10;      
 int i;        
 a=(int*)malloc(sizeof(int)*N);  
 d=(int*)malloc(sizeof(int)*1);            
 for(i=0;i<N;i++)       
 {       
     a[i]=i;        
 }        
 cudaMalloc((void**)&d_a,sizeof(int)*N);        
cudaMalloc((void**)&l,sizeof(int)*1);        
 cudaMemcpy(d_a,a,sizeof(int)*N,cudaMemcpyHostToDevice);            
 find<<<2,5>>>(d_a,l,5);        
 cudaMemcpy(d,l,sizeof(int)*1,cudaMemcpyDeviceToHost);        
printf("output of 18bce1005 sruti verma\n");    
 printf("Success\n");        
     printf("%d\n",d[0]);          
 cudaFree(d_a);        
  cudaFree(l);        
 free(a);            
 return 0;        
}        

output of 18bce1005 sruti verma
Success
5



In [ ]:
%%cu 
#include<stdio.h> 
// generating a matrix
__global__ void matrixFill ( int *x ) 
{  
int bx = blockIdx.x; 
int by = blockIdx.y; 
int tx = threadIdx.x; 
int ty = threadIdx.y; 
int col = by*blockDim.y + ty;  
int row = bx*blockDim.x + tx;  
int dim =blockDim.x*gridDim.x; 
int i = row*dim + col;  
x[i] = i; 
}  
int main ( int argc, char* argv[] ) 
{  
const int xb = 2; /* gridDim.x */ 
const int yb = 2; /* gridDim.y */ 
const int zb = 1; /* gridDim.z */ 
const int xt = 2; /* blockDim.x */ 
const int yt = 2; /* blockDim.y */ 
const int zt = 1; /* blockDim.z */ 
const int n = xb*yb*zb*xt*yt*zt; 
printf("output of 18bce1005 sruti verma\n");
printf("allocating array of length %d...\n",n); 
int *xhost = (int*)calloc(n,sizeof(int)); 
for(int i=0; i<n; i++) xhost[i] = -1.0; 
int *xdevice; 
size_t sx = n*sizeof(int); 
cudaMalloc((void**)&xdevice,sx); 
cudaMemcpy(xdevice,xhost,sx,cudaMemcpyHostToDevice); 
dim3 dimGrid(xb,yb,zb); 
dim3 dimBlock(xt,yt,zt); 
matrixFill<<<dimGrid,dimBlock>>>(xdevice); 
cudaMemcpy(xhost,xdevice,sx,cudaMemcpyDeviceToHost); 
cudaFree(xdevice); 
int *p = xhost; 
for(int i1=0; i1 < xb; i1++) 
for(int i2=0; i2 < yb; i2++) 
for(int i3=0; i3 < zb; i3++) 
for(int i4=0; i4 < xt; i4++) 
for(int i5=0; i5 < yt; i5++) 
for(int i6=0; i6 < zt; i6++) 
printf("x[%d][%d][%d][%d][%d][%d] = %d\n",i1,i2,i3,i4,i5,i6,*(p++)); 
return 0; 
}  

output of 18bce1005 sruti verma
allocating array of length 16...
x[0][0][0][0][0][0] = 0
x[0][0][0][0][1][0] = 1
x[0][0][0][1][0][0] = 2
x[0][0][0][1][1][0] = 3
x[0][1][0][0][0][0] = 4
x[0][1][0][0][1][0] = 5
x[0][1][0][1][0][0] = 6
x[0][1][0][1][1][0] = 7
x[1][0][0][0][0][0] = 8
x[1][0][0][0][1][0] = 9
x[1][0][0][1][0][0] = 10
x[1][0][0][1][1][0] = 11
x[1][1][0][0][0][0] = 12
x[1][1][0][0][1][0] = 13
x[1][1][0][1][0][0] = 14
x[1][1][0][1][1][0] = 15



In [ ]:
%%cu
#include<stdio.h> 
// sum of two matrices using simple method
__global__ void matadd(int *l,int *m, int *n) 
{ 
    int x=blockIdx.x; 
    int y=blockIdx.y; 
    int id=gridDim.x * y +x; 
    n[id]=l[id]+m[id]; 
} 
int main() 
{ 
    int a[2][3]; 
    int b[2][3]; 
    int c[2][3]; 
    int *d,*e,*f; 
    int i,j; 
    for(i=0;i<2;i++) 
    { 
        for(j=0;j<3;j++) 
            { 
                a[i][j]=i+j; 
            } 
    } 
        for(i=0;i<2;i++) 
        { 
            for(j=0;j<3;j++) 
                { 
                   b[i][j]=i; 
                } 
        } 
    cudaMalloc((void **)&d,2*3*sizeof(int)); 
    cudaMalloc((void **)&e,2*3*sizeof(int)); 
    cudaMalloc((void **)&f,2*3*sizeof(int)); 
 cudaMemcpy(d,a,2*3*sizeof(int),cudaMemcpyHostToDevice); 
 cudaMemcpy(e,b,2*3*sizeof(int),cudaMemcpyHostToDevice); 
     
dim3 grid(3,2); 
/* Here we are defining two dimensional Grid(collection of blocks) structure. Syntax is dim3 grid(no. of columns,no. of rows) */ 
 
    matadd<<<grid,1>>>(d,e,f); 
 
 cudaMemcpy(c,f,2*3*sizeof(int),cudaMemcpyDeviceToHost); 
    printf("\nSum of two matrices:\n "); 
    for(i=0;i<2;i++) 
    { 
        for(j=0;j<3;j++) 
        { 
              printf("%d\t",c[i][j]); 
        } 
        printf("\n"); 
    } 
    cudaFree(d); 
    cudaFree(e); 
    cudaFree(f); 
    return 0; 
} 


 Enter elements of first matrix of size 2 * 3

 Enter elements of second matrix of size 2 * 3

Sum of two matrices:
 0	1	2	
2	3	4	



In [ ]:
%%cu
#include<stdio.h> 
// transpose of matrix using simple method(only for square matrices)
__global__ void matadd(int *l,int *m) 
{ 
    int x=blockIdx.x; 
    int y=blockIdx.y; 
    int id=gridDim.x * y +x; 
    int id1=gridDim.y * x +y; 
 m[id1]=l[id];
} 
int main() 
{ 
    int a[3][2];
    int c[2][3]; 
    int *d,*f; 
    int i,j; 
    for(i=0;i<3;i++) 
    { 
        for(j=0;j<2;j++) 
            { 
                a[i][j]=i; 
            } 
    } 
    cudaMalloc((void **)&d,3*2*sizeof(int)); 
    cudaMalloc((void **)&f,3*2*sizeof(int)); 
 cudaMemcpy(d,a,3*2*sizeof(int),cudaMemcpyHostToDevice); 
     dim3 grid(3,2); 
    matadd<<<grid,1>>>(d,f); 
cudaMemcpy(c,f,3*2*sizeof(int),cudaMemcpyDeviceToHost); 
    printf("\nTranspose:\n "); 
    for(i=0;i<2;i++) 
    { 
        for(j=0;j<3;j++) 
        { 
              printf("%d\t",c[i][j]); 
        } 
        printf("\n"); 
    } 
    cudaFree(d); 
    cudaFree(f); 
    return 0; 
} 


Transpose:
 0	1	0	
2	1	2	



In [ ]:
%%cu
#include<stdio.h> 
#define row1 2 /* Number of rows of first matrix */ 
#define col1 3 /* Number of columns of first matrix */ 
#define row2 3 /* Number of rows of second matrix */ 
#define col2 2 /* Number of columns of second matrix */ 

// product of two matrices  using simple method
__global__ void matproduct(int *l,int *m, int *n) 
{ 
    int x=blockIdx.x; 
    int y=blockIdx.y; 
    int k; 
   
n[col2*y+x]=0; 
for(k=0;k<col1;k++) 
   { 
    n[col2*y+x]=n[col2*y+x]+l[col1*y+k]*m[col2*k+x]; 
   } 
} 
 
int main() 
{ 
    int a[row1][col1]; 
    int b[row2][col2]; 
    int c[row1][col2]; 
    int *d,*e,*f; 
    int i,j; 
 
    printf("\n Enter elements of first matrix of size 2*3\n"); 
    for(i=0;i<row1;i++) 
    { 
        for(j=0;j<col1;j++) 
            { 
                a[i][j]=i; 
            } 
    } 
    printf("\n Enter elements of second matrix of size 3*2\n"); 
        for(i=0;i<row2;i++) 
        { 
            for(j=0;j<col2;j++) 
                { 
                   b[i][j]=i; 
                } 
        } 
 
    cudaMalloc((void **)&d,row1*col1*sizeof(int)); 
    cudaMalloc((void **)&e,row2*col2*sizeof(int)); 
    cudaMalloc((void **)&f,row1*col2*sizeof(int)); 
 
 cudaMemcpy(d,a,row1*col1*sizeof(int),cudaMemcpyHostToDevice); 
 cudaMemcpy(e,b,row2*col2*sizeof(int),cudaMemcpyHostToDevice); 
 
dim3 grid(col2,row1); 
/* Here we are defining two dimensional Grid(collection of blocks) structure. Syntax is dim3 grid(no. of columns,no. of rows) */ 
 
    matproduct<<<grid,1>>>(d,e,f); 
 
 cudaMemcpy(c,f,row1*col2*sizeof(int),cudaMemcpyDeviceToHost); 
    printf("\nProduct of two matrices:\n "); 
    for(i=0;i<row1;i++) 
    { 
        for(j=0;j<col2;j++) 
        { 
              printf("%d\t",c[i][j]); 
        } 
        printf("\n"); 
    } 
 
    cudaFree(d); 
    cudaFree(e); 
    cudaFree(f); 
 
    return 0; 
} 


 Enter elements of first matrix of size 2*3

 Enter elements of second matrix of size 3*2

Product of two matrices:
 0	0	
3	3	



In [ ]:
%%cu 
#include <stdio.h> 
// square of each element of matrix 
__global__ void matrixFill ( int *x ) 
{ 
int bx = blockIdx.x; 
int by = blockIdx.y; 
int tx = threadIdx.x; 
int ty = threadIdx.y; 
int col = by*blockDim.y + ty;  
int row = bx*blockDim.x + tx;  
int dim =blockDim.x*gridDim.x; 
int i = row*dim + col;  
x[i] = i; 
} 
__global__ void matrixsquare (int *x) 
{ 
int bx = blockIdx.x; 
int by = blockIdx.y; 
int tx = threadIdx.x; 
int ty = threadIdx.y; 
int col = by*blockDim.y + ty;  
int row = bx*blockDim.x + tx;  
int dim =blockDim.x*gridDim.x; 
int i = row*dim + col;  
x[i] = x[i]*x[i]; 
} 
int main ( int argc, char* argv[] ) 
{ 
const int xb = 2; 
const int yb = 2; 
const int zb = 1;  
const int xt = 2;  
const int yt = 2; 
const int zt = 1; 
const int n = xb*yb*zb*xt*yt*zt; 
printf("allocating array of length %d...\n",n); 
int *xhost = (int*)calloc(n,sizeof(int)); 
for(int i=0; i<n; i++) xhost[i] = -1.0; 
int *xdevice; 
size_t sx = n*sizeof(int); 
cudaMalloc((void**)&xdevice,sx); 
cudaMemcpy(xdevice,xhost,sx,cudaMemcpyHostToDevice); 
dim3 dimGrid(xb,yb,zb); 
dim3 dimBlock(xt,yt,zt);
matrixFill<<<dimGrid,dimBlock>>>(xdevice); 
matrixsquare<<<dimGrid,dimBlock>>>(xdevice); 
cudaMemcpy(xhost,xdevice,sx,cudaMemcpyDeviceToHost); 
cudaFree(xdevice); 
printf("output of 18bce1005 sruti verma\n"); 
int *p = xhost; 
for(int i1=0; i1 < xb; i1++) 
for(int i2=0; i2 < yb; i2++) 
for(int i3=0; i3 < zb; i3++) 
for(int i4=0; i4 < xt; i4++) 
for(int i5=0; i5 < yt; i5++) 
for(int i6=0; i6 < zt; i6++) 
printf("x[%d][%d][%d][%d][%d][%d] = %d\n",i1,i2,i3,i4,i5,i6,*(p++)); 
return 0; 
} 

allocating array of length 16...
output of 18bce1005 sruti verma
x[0][0][0][0][0][0] = 0
x[0][0][0][0][1][0] = 1
x[0][0][0][1][0][0] = 4
x[0][0][0][1][1][0] = 9
x[0][1][0][0][0][0] = 16
x[0][1][0][0][1][0] = 25
x[0][1][0][1][0][0] = 36
x[0][1][0][1][1][0] = 49
x[1][0][0][0][0][0] = 64
x[1][0][0][0][1][0] = 81
x[1][0][0][1][0][0] = 100
x[1][0][0][1][1][0] = 121
x[1][1][0][0][0][0] = 144
x[1][1][0][0][1][0] = 169
x[1][1][0][1][0][0] = 196
x[1][1][0][1][1][0] = 225



In [ ]:
%%cu   
#include<stdio.h>   
// matrix transpose 
__global__ void matrixFill ( int *x )   
{    
int bx = blockIdx.x;   
int by = blockIdx.y;   
int tx = threadIdx.x;   
int ty = threadIdx.y;   
int col = by*blockDim.y + ty;    
int row = bx*blockDim.x + tx;    
int dim =blockDim.x*gridDim.x;   
int i = row*dim + col;    
x[i] = i;   
}    
__global__ void transpose ( int *y,int *x )   
{   
int bx = blockIdx.x;   
int by = blockIdx.y;   
int tx = threadIdx.x;   
int ty = threadIdx.y;   
int col = by*blockDim.y + ty;    
int row = bx*blockDim.x + tx;    
int dim =blockDim.x*gridDim.x;   
int i = row*dim + col;    
int j = col*dim + row;    
y[j] = x[i];   
}  
int main ( int argc, char* argv[] )   
{    
const int xb = 2; /* gridDim.x */   
const int yb = 2; /* gridDim.y */   
const int zb = 1; /* gridDim.z */   
const int xt = 2; /* blockDim.x */   
const int yt = 2; /* blockDim.y */   
const int zt = 1; /* blockDim.z */   
const int n = xb*yb*zb*xt*yt*zt;   
printf("output of 18bce1005 sruti verma\n");
printf("allocating array of length %d...\n",n);   
int *xhost = (int*)calloc(n,sizeof(int));   
int *yhost = (int*)calloc(n,sizeof(int));      
for(int i=0; i<n; i++) xhost[i] = -1.0;   
for(int i=0; i<n; i++) yhost[i] = -1.0;    
int *xdevice; int *ydevice;  
size_t sx = n*sizeof(int);   
cudaMalloc((void**)&xdevice,sx);  
size_t sy = n*sizeof(int);   
cudaMalloc((void**)&ydevice,sy);    
cudaMemcpy(xdevice,xhost,sx,cudaMemcpyHostToDevice);   
cudaMemcpy(ydevice,yhost,sy,cudaMemcpyHostToDevice);  
dim3 dimGrid(xb,yb,zb);   
dim3 dimBlock(xt,yt,zt);   
matrixFill<<<dimGrid,dimBlock>>>(xdevice);   
cudaMemcpy(xhost,xdevice,sx,cudaMemcpyDeviceToHost);   
int *p = xhost;   
for(int i1=0; i1 < xb; i1++)   
for(int i2=0; i2 < yb; i2++)   
for(int i3=0; i3 < zb; i3++)   
for(int i4=0; i4 < xt; i4++)   
for(int i5=0; i5 < yt; i5++)   
for(int i6=0; i6 < zt; i6++)   
printf("x[%d][%d][%d][%d][%d][%d] = %d\n",i1,i2,i3,i4,i5,i6,*(p++));   
transpose<<<dimGrid,dimBlock>>>(ydevice,xdevice);   
cudaMemcpy(yhost,ydevice,sy,cudaMemcpyDeviceToHost);   
int *q = yhost;   
for(int i1=0; i1 < xb; i1++)   
for(int i2=0; i2 < yb; i2++)   
for(int i3=0; i3 < zb; i3++)   
for(int i4=0; i4 < xt; i4++)   
for(int i5=0; i5 < yt; i5++)   
for(int i6=0; i6 < zt; i6++)   
printf("y[%d][%d][%d][%d][%d][%d] = %d\n",i1,i2,i3,i4,i5,i6,*(q++));   
cudaFree(ydevice);  
cudaFree(xdevice);  
return 0;   
}    

output of 18bce1005 sruti verma
allocating array of length 16...
x[0][0][0][0][0][0] = 0
x[0][0][0][0][1][0] = 1
x[0][0][0][1][0][0] = 2
x[0][0][0][1][1][0] = 3
x[0][1][0][0][0][0] = 4
x[0][1][0][0][1][0] = 5
x[0][1][0][1][0][0] = 6
x[0][1][0][1][1][0] = 7
x[1][0][0][0][0][0] = 8
x[1][0][0][0][1][0] = 9
x[1][0][0][1][0][0] = 10
x[1][0][0][1][1][0] = 11
x[1][1][0][0][0][0] = 12
x[1][1][0][0][1][0] = 13
x[1][1][0][1][0][0] = 14
x[1][1][0][1][1][0] = 15
y[0][0][0][0][0][0] = 0
y[0][0][0][0][1][0] = 4
y[0][0][0][1][0][0] = 8
y[0][0][0][1][1][0] = 12
y[0][1][0][0][0][0] = 1
y[0][1][0][0][1][0] = 5
y[0][1][0][1][0][0] = 9
y[0][1][0][1][1][0] = 13
y[1][0][0][0][0][0] = 2
y[1][0][0][0][1][0] = 6
y[1][0][0][1][0][0] = 10
y[1][0][0][1][1][0] = 14
y[1][1][0][0][0][0] = 3
y[1][1][0][0][1][0] = 7
y[1][1][0][1][0][0] = 11
y[1][1][0][1][1][0] = 15



In [ ]:
%%cu  
#include<stdio.h>  
// transpose and subtracting 2 matrices
__global__ void transpose(int *l,int *m)  
{  
int x=blockIdx.x;  
int y=blockIdx.y;  
int id=gridDim.x * y +x;  
int id1=gridDim.y * x +y;  
m[id1]=l[id];  
}  
__global__ void matsub(int *l,int *m, int *n)  
{  
int x=blockIdx.x;  
int y=blockIdx.y;  
int id=gridDim.x * y +x;  
n[id]=l[id]-m[id];  
}  
int main()  
{ 
int a[3][3];  
int t[3][3];  
int res[3][3];  
int *d,*e,*f; int i,j;  
a[0][0]=9;  
a[0][1]=9;  
a[0][2]=9;  
a[1][0]=4;  
a[1][1]=7;  
a[1][2]=0;  
a[2][0]=9;  
a[2][1]=3;  
a[2][2]=0;  
cudaMalloc((void **)&d,3*3*sizeof(int));  
cudaMalloc((void **)&f,3*3*sizeof(int));  
cudaMemcpy(d,a,3*3*sizeof(int),cudaMemcpyHostToDevice);  
dim3 grid(3,3);  
transpose<<<grid,1>>>(d,f);  
cudaMemcpy(t,f,3*3*sizeof(int),cudaMemcpyDeviceToHost);  
printf("Original matrix\n");  
for(i=0;i<3;i++)  
{  
for(j=0;j<3;j++)  
{  
printf("%d\t",a[i][j]);  
}  
printf("\n");  
}  
printf("\nTranspose:\n ");  
for(i=0;i<3;i++)  
{  
for(j=0;j<3;j++)  
{  
printf("%d\t",t[i][j]);  
}  
printf("\n");  
}  
cudaFree(d);  
cudaFree(f);  
cudaMalloc((void **)&d,3*3*sizeof(int));  
cudaMalloc((void **)&e,3*3*sizeof(int));  
cudaMalloc((void **)&f,3*3*sizeof(int));  
cudaMemcpy(d,a,3*3*sizeof(int),cudaMemcpyHostToDevice);  
cudaMemcpy(e,t,3*3*sizeof(int),cudaMemcpyHostToDevice);  
matsub<<<grid,1>>>(d,e,f);  
cudaMemcpy(res,f,3*3*sizeof(int),cudaMemcpyDeviceToHost);  
printf("\nSubtraction of two matrices:\n ");  
for(i=0;i<3;i++)  
{  
for(j=0;j<3;j++) {  
printf("%d\t",res[i][j]);  
}  
printf("\n");  
}  
cudaFree(d);  
cudaFree(e);  
cudaFree(f);  
return 0;  
}   

Original matrix
9	9	9	
4	7	0	
9	3	0	

Transpose:
 9	4	9	
9	7	3	
9	0	0	

Subtraction of two matrices:
 0	5	0	
-5	0	-3	
0	3	0	



In [ ]:
%%cu
// Using CUDA device to calculate pi
#include <stdio.h>


#define NBIN  10000000  // Number of bins
#define NUM_BLOCK   13  // Number of thread blocks
#define NUM_THREAD 192  // Number of threads per block
int tid;
float pi = 0;

// Kernel that executes on the CUDA device
__global__ void cal_pi(float *sum, int nbin, float step, int nthreads, int nblocks) {
	int i;
	float x;
	int idx = blockIdx.x*blockDim.x+threadIdx.x;  // Sequential thread index across the blocks
	for (i=idx; i< nbin; i+=nthreads*nblocks) {
		x = (i+0.5)*step;
		sum[idx] += 4.0/(1.0+x*x);
	}
}

// Main routine that executes on the host
int main(void) {
	dim3 dimGrid(NUM_BLOCK,1,1);  // Grid dimensions
	dim3 dimBlock(NUM_THREAD,1,1);  // Block dimensions
	float *sumHost, *sumDev;  // Pointer to host & device arrays

	float step = 1.0/NBIN;  // Step size
	size_t size = NUM_BLOCK*NUM_THREAD*sizeof(float);  //Array memory size
	sumHost = (float *)malloc(size);  //  Allocate array on host
	cudaMalloc((void **) &sumDev, size);  // Allocate array on device
	// Initialize array in device to 0
	cudaMemset(sumDev, 0, size);
	// Do calculation on device
	cal_pi <<<dimGrid, dimBlock>>> (sumDev, NBIN, step, NUM_THREAD, NUM_BLOCK); // call CUDA kernel
	// Retrieve result from device and store it in host array
	cudaMemcpy(sumHost, sumDev, size, cudaMemcpyDeviceToHost);
	for(tid=0; tid<NUM_THREAD*NUM_BLOCK; tid++)
		pi += sumHost[tid];
	pi *= step;

	// Print results
	printf("PI = %f\n",pi);

	// Cleanup
	free(sumHost); 
	cudaFree(sumDev);

	return 0;
}